In [1]:
import pandas as pd
import cv2
from transformers import CLIPProcessor, CLIPModel
import os
import ast
from PIL import Image
from deep_translator import GoogleTranslator, DeeplTranslator
import matplotlib.pyplot as plt

c:\Users\fanti\miniconda3\envs\tiamat_environnement\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Extraction des images détectées par le Yolo dans des fichiers séparés ##

In [12]:
def extract_images(fichier, chemin_image):
    df=pd.read_csv(fichier)
    liste_coords =df['box'].to_list()
        
    path=os.getcwd()
    
    image = cv2.imread(path + "\\"+ chemin_image)
    
    if image is None:
            print(f"Erreur : impossible de charger l'image {chemin_image}")
            exit()
    nom_dossier = chemin_image.replace(".jpg","").replace('images_a_traiter\\','')
    print(nom_dossier)
    if not os.path.exists(path+"\\"+"sorties"+"\\"+nom_dossier):
        os.makedirs("sorties\\"+nom_dossier)
        print(f"{nom_dossier} créé")
    compteur=0
    for i in liste_coords:
        compteur+=1
        dico = ast.literal_eval(i)
        valeurs=list(dico.values())
        
        y1, x1, y2, x2 = int(round(valeurs[0])), int(round(valeurs[1])), int(round(valeurs[2])), int(round(valeurs[3]))
        cropped_img= image[x1:x2,y1:y2]
        cv2.imwrite(f"sorties\\{nom_dossier}\\{nom_dossier}_illus{compteur}.png", cropped_img)
            


In [ ]:
path=os.getcwd()+"\\images_a_traiter"
for root, dirs, file in os.walk(path):
    for image in file:
        if ".jpg" in image:
            nom_csv = image+"_annots.csv"
            chemin_image = "images_a_traiter\\"+image
            extract_images(nom_csv, chemin_image)

Fonction optionnelle pour afficher les images d'une page sous forme de grille, empruntée à HALUKPA Kerry, "Getting started with OpenAI's CLIP", sur Medium, 2023, consulté le 27/08/2025.
https://medium.com/@kerry.halupka/getting-started-with-openais-clip-a3b8f5277867

In [2]:

def image_grid(imgs, cols):
    rows = (len(imgs) + cols - 1) // cols
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


def montrer_images(nom_image):
    path=os.getcwd()

    image_urls=[]
    for i in range(1,20):
        image_urls.append(str(path+"\\"+"sorties"+"\\"+nom_image+"_illus"+str(i)+".png"))
    images = []
    try:
        for url in image_urls:
            images.append(Image.open(url))
    except:
        pass

    grid = image_grid(images, cols=3)
    display(grid)

In [3]:
def clip_integral():
    liste_images = obtenir_liste_images()
    for i in liste_images:
        liste2=obtenir_liste_captions(i)
        liste3 = traduire(liste2)
        
        application_CLIP(liste3, i, liste2)

In [4]:
def obtenir_liste_images():
    path=os.getcwd()+"\\sorties"
    liste_images = []
    for root, dirs, files in os.walk(path):
            
        for dir in dirs:
                
            liste_images.append(str(dir))
    return(liste_images)

In [5]:
def obtenir_liste_captions(nom_image):
   fichier_texte=str('dossier_output_experience\\texte'+nom_image+".jpg.txt") #modifier le .jpg selon le format de l'image de la page de journal originale
   with open (fichier_texte,'r', encoding="utf-8") as ft:
         texte=ft.read()
         liste_zones=texte.split("Zone_texte")
         
         liste2=[]
         for i in range(0, len(liste_zones)):
            
            if len(liste_zones[i])>= 10 and len(liste_zones[i])<=77:
               terme=liste_zones[i].replace("\n","")
               terme=liste_zones[i].replace("- ","")
               liste2.append(terme)
   return(liste2)       


In [6]:
def traduire(liste2):
    # liste3= DeeplTranslator(api_key=" INPUT YOUR OWN API KEY", source="fr", target="en", use_free_api=True).translate_batch(liste2)
    liste3= GoogleTranslator("fr","en").translate_batch(liste2)
    return(liste3)
#Cellule pour passer les textes en anglais, CLIP ayant été entraîné sur des textes en anglais.

In [7]:
def application_CLIP(liste3, dossier, liste2):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    images=[]
    path= os.getcwd()+"\\sorties"+"\\"+dossier
    for root, dirs, files in os.walk(path):
        for image in files:
            if ".png" in image:
                image= Image.open(path+"\\"+image)
                images.append(image)
        
    inputs = processor(text=liste3, images=images, return_tensors="pt", padding=True, do_rescale= True, do_convert_rgb=False)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
    liste_probas =probs.tolist() 
    
    liste_probas_par_ordre_images=[]
    for i in range(len(liste_probas)):
        indices_probas = []
        for idx, probab in enumerate(liste_probas[i]):  # Use enumerate to get index
            if probab >= 0.2:
                indices_probas.append(idx)  # Use the actual index from enumerate
        liste_probas_par_ordre_images.append(indices_probas)
    compteur=1
    print(liste_probas_par_ordre_images)
    for i in liste_probas_par_ordre_images:
        with open(f"{dossier}.txt","a")as file:
            file.write(f"image n{compteur}\n")
            for y in i:
                file.write(liste2[y])
                
        compteur+=1
            
        
    return(probs, liste_probas_par_ordre_images)        

In [8]:
clip_integral()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


[[9, 28], [9, 22], [9], [9, 22], [9], [9], [9, 22], [9], [9, 28], [9, 28]]
[[12], [12], [10, 12], [1, 14], [1]]
